In [ ]:
#connect with Azure Gen2 data lake and mount it on Databricks
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": "XXXXXXXXXXXXXXXXX",
       "fs.azure.account.oauth2.client.secret": "XXXXXXXXXXXXX",
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/XXXXXXXXXXXXXX/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}
dbutils.fs.mount(
source = "abfss://XXXX@dydstorage2021gen2.dfs.core.windows.net/",
mount_point = "/mnt/newfolder",
extra_configs = configs)

In [ ]:
# List all the mounted folders 
display(dbutils.fs.mounts())

In [ ]:
#List all files within a specific folder
filepath=('/mnt/xxxx/')
dbutils.fs.ls(filepath)

In [ ]:
#load all the files within the folde with specific name format
df = (spark.read.csv(filepath+"xxxxx"+"*", header="true", inferSchema="true"))
# - First line of file is a header
# - Automatically infer the schema of the data

In [ ]:
#shape of the dataframe
print((df.count(), len(df.columns)))

In [ ]:
#view dataframe
df.show()

In [ ]:
#view and download data if required
display(df)

In [ ]:
#Count any variable, for example date 
df.count("Date").show()
#frequency count for any variable 
pd.Series(df["totalsensors"]).value_counts()

In [ ]:
#perform data wrangling####
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date
from pyspark.sql.functions import *
import pyspark
df.select(to_timestamp(df.date_time, 'yyyy-MM-dd HH:mm:ss').alias('DateTime')).collect()
df=df.withColumn("DateTime",to_timestamp("date_time"))
df=df.withColumn("Date",to_date("DateTime"))
df=df.toPandas()
df=df.select(col("Date"), to_date(col("DateTime")).alias("to_date"))
df_before=df.filter(df.Date > "2020-03-18")
df_during=df.filter(df.Date <= "2020-03-18")
df_before.show()
df_during.show()

In [ ]:
#Write result datafarme to storage on Azure
df_before.write \
.format("jdbc")\
.option("url", "jdbc:sqlserver://xxxx-data.database.windows.net:1433;database=xxxx")\
.option("dbtable", "dbo.ab_df")\
.option("user", "xxxx@uxxx.ca")\
.option("password", "xxxxx")\
.save()

In [ ]:
#save result as table on Azure
df_before.write.option("header",True) \
        .partitionBy("Identifier") \
        .mode("overwrite") \
        .csv("/mnt/xxxx/")

In [ ]:
#save result as table on databricks
df.write.saveAsTable("df")

In [ ]:
#load data from databricks table
df = spark.sql("select * from df")